Code for generating the neighbour and greedy neighbour orderings

In [5]:
load("utils.sage")

def get_neighbour_ordering(G, start_set):
    ordered_vertices = copy(start_set)
    left_to_add_neighbors = copy(start_set)
    while len(ordered_vertices) < len(G.vertices()):
        expand_vertex = left_to_add_neighbors[0]
        left_to_add_neighbors.remove(expand_vertex)
        for n in G.neighbors(expand_vertex):
            if n not in ordered_vertices:
                ordered_vertices.append(n)
                left_to_add_neighbors.append(n)
    return ordered_vertices

def get_greedy_ordering(G, start_set):

    def phi(G, C):
        C_comp = vertex_complement(G, C)
        C_expansion = edge_expansion(G, C)
        C_comp_expansion = edge_expansion(G, C_comp)
        return max(C_expansion, C_comp_expansion)

    ordered_vertices = copy(start_set)
    cut_neighbors = []
    for v in ordered_vertices:
        for n in G.neighbors(v):
            if n not in cut_neighbors:
                cut_neighbors.append(n)
    while len(ordered_vertices) < len(G.vertices()) - 1:
        # iterate over cut_neighbors and add one which gives smallest edge expansion
        minnum = 2
        next_neighbor = None
        for neighbor in cut_neighbors:
            cut = copy(ordered_vertices)
            cut.append(neighbor)
            expansion = phi(G, cut)
            if expansion < minnum:
                minnum = expansion
                next_neighbor = neighbor    
        # add it to ordered_vertices
        ordered_vertices.append(next_neighbor)
        # add its neighbors to cut_neighbors, and remove it from cut_neighbors
        cut_neighbors.remove(next_neighbor)
        for n in G.neighbors(next_neighbor):
            if n not in cut_neighbors:
                if n not in ordered_vertices:
                    cut_neighbors.append(n)
    # Add last vertex
    for v in G.vertices():
        if v not in ordered_vertices:
            ordered_vertices.append(v)
            break
    return ordered_vertices

We now give an example of using the sweep cut based fiedlers algorithm for these orderings.

In [19]:
load("utils.sage")

# Pick parameters
p = 401
L = [2,3]

# Get L isogeny graph
G = get_L_graph(p, L)

# Get a random staring vertex for neighbour and greedy neighbour orderings
start_vertex = G.vertices()[randrange(0, len(G.vertices()))]

# Get different orderings
fiedlers_ordering = get_spectral_ordering(G, use_numpy_over_sage=False)
neighbour_ordering = get_neighbour_ordering(G, [start_vertex])
greedy_ordering = get_greedy_ordering(G, [start_vertex])

# Apply the algorithm to find cut with smallest expansion
exp1, cut1 = fiedler(G, ordering=fiedlers_ordering)
print("Spectral ordering. Expansion: " + str(exp1) + "     Cut size: " + str(len(cut1)))
exp2, cut2 = fiedler(G, ordering=neighbour_ordering)
print("Neighbour ordering. Expansion: " + str(exp2) + "     Cut size: " + str(len(cut2)))
exp3, cut3 = fiedler(G, ordering=greedy_ordering)
print("Greedy ordering. Expansion: " + str(exp3) + "      Cut size: " + str(len(cut3)))


Spectral ordering. Expansion: 0.46218487394957986     Cut size: 17
Neighbour ordering. Expansion: 0.44537815126050423     Cut size: 17
Greedy ordering. Expansion: 0.2773109243697479      Cut size: 17


We run this to produce a comparison table. We use 3 repetitions (starting curves) for neighbour and greedy orderings, and take the average.

In [7]:
param_sets = [
    # prime,  L
    (419,   [3]),
    (419,   [2,3]),
    (419, [2,3,5,7,11]),
    (5569, [3]),
    (5569, [2,3]),
    (5569, [2,3,5,7,11]),
    (10007, [3])
]

repetitions = 3

for param_set in param_sets:
    p, L = param_set
    G = get_L_graph(p, L)

    fiedlers_expansion = fiedler(G, use_numpy_over_sage=True)[0]

    neighbour_expansions = []
    greedy_expansions = []
    for r in range(0, repetitions):
        start_vertex = G.vertices()[randrange(0, len(G.vertices()))]
        neighbour_expansions.append(fiedler(G, ordering=get_neighbour_ordering(G, [start_vertex]))[0])
        greedy_expansions.append(fiedler(G, ordering=get_greedy_ordering(G, [start_vertex]))[0])
    avg_neighbour = sum(neighbour_expansions) / repetitions
    avg_greedy = sum(greedy_expansions) / repetitions

    print("")
    print((p, L))
    print("Fiedler: " + str(fiedlers_expansion))
    print("Neighbour: " + str(avg_neighbour))
    print("Greedy: " + str(avg_greedy))


(419, [3])
Fiedler: 0.5972222222222222
Neighbour: 0.31895424836601305
Greedy: 0.22487745098039214

(419, [2, 3])
Fiedler: 0.4523809523809524
Neighbour: 0.28908128908128905
Greedy: 0.28571428571428575

(419, [2, 3, 5, 7, 11])
Fiedler: 0.4882154882154882
Neighbour: 0.49102132435465773
Greedy: 0.4126889813164323

(5569, [3])
Fiedler: 0.49353448275862066
Neighbour: 0.3007134149375529
Greedy: 0.17816091954022992



(5569, [2, 3])
Fiedler: 0.4839901477832512
Neighbour: 0.32215007215007213
Greedy: 0.19518455058185194

(5569, [2, 3, 5, 7, 11])
Fiedler: 0.49738766980146293
Neighbour: 0.47300592128178326
Greedy: 0.34308603274120514



(10007, [3])
Fiedler: 0.4885817307692308
Neighbour: 0.28702724567334087
Greedy: 0.16966426858513187
